In [3]:
#importing the libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import pprint, time
import random
from sklearn.model_selection import train_test_split

In [4]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'MyDrive'
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
'10 22 2021.gdoc'
 136817BC-0A78-4D48-AE08-A84D3CA61EB9.jpeg
'An Intelligent System for Waste Materials Segregation Using IoT and Deep Learning.pdf'
 baby.jpeg
 batches.meta
 bigdata/
 bigdata_project.gslides
 breast-cancer.data
'car (1).data'
 checkmodel.h5
 CN2_screenshot.gdoc
 CN_Homework1.gdoc
'CN_Homework 2.gdoc'
 CN_homework3.gdoc
 CN_pipeline.gdoc
'Colab Notebooks'/
'COMPUTER VISION ASSIGNMENT 3.gdoc'
'Copy of Pepsi and Vending Refund Request.gform'@
'Copy of Pepsi and Vending Refund Request OLD.gform'
'Copy of Pepsi and Vending Refunds (Responses) OLD.gsheet'
'Copy of Student Development Bank Deposit Form.gform'@
'CSCI 4930 - BIG DATA ANALYTICS.gdoc'
'CSO Reimbursement Request Form - Pending.gsheet'
'CV - 0121.gdoc'
 CV_FinalProject.gdoc
 data_batch_1
 data_batch_2
 data_batch_3
 data_batch_4
 data_batch_5
'Data Science Notes.gd

In [5]:
#load the datasets
data_ht=pd.read_csv('train_ht.tsv',sep='\t',header=None)
data_ht.columns = ['words','tag']
data_ht

,words,tag
0,konsiltan,k-on-s-i-l-t-an
1,depotwa,d-e-p-o-t-w-a
2,sosyopwofesyonèl,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l
3,vejetal,v-e-j-e-t-a-l
4,repibliye,r-e-p-i-b-l-i-y-e
...,...,...
12807,Remi,R-e-m-i
12808,diskriminatwa,d-i-s-k-r-i-m-i-n-a-t-w-a
12809,rejè,r-e-j-è
12810,manzè,m-an-z-è


In [6]:
#convert to lowercase
data_ht['tag'] = data_ht['tag'].str.lower()
data_ht

,words,tag
0,konsiltan,k-on-s-i-l-t-an
1,depotwa,d-e-p-o-t-w-a
2,sosyopwofesyonèl,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l
3,vejetal,v-e-j-e-t-a-l
4,repibliye,r-e-p-i-b-l-i-y-e
...,...,...
12807,Remi,r-e-m-i
12808,diskriminatwa,d-i-s-k-r-i-m-i-n-a-t-w-a
12809,rejè,r-e-j-è
12810,manzè,m-an-z-è


In [7]:
#converting to lists
word_list=data_ht['words'].to_list()
tag_list=data_ht['tag'].to_list()

In [8]:
tag_list[505].split('-')

['k', 'a', 't', 'ch', 'ou', 'b', 'ou', 'm', 'b', 'e']

In [20]:
a = []
for i in tag_list:
  i = i.split('-')
  list1 = []
  for j in i:
    list1.append((j[0], 'B'))
    if len(j) > 1:
      for k in j[1:]:
        list1.append((k, 'I'))
  a.append(list1)

In [21]:
a

[[('k', 'B'),
  ('o', 'B'),
  ('n', 'I'),
  ('s', 'B'),
  ('i', 'B'),
  ('l', 'B'),
  ('t', 'B'),
  ('a', 'B'),
  ('n', 'I')],
 [('d', 'B'),
  ('e', 'B'),
  ('p', 'B'),
  ('o', 'B'),
  ('t', 'B'),
  ('w', 'B'),
  ('a', 'B')],
 [('s', 'B'),
  ('o', 'B'),
  ('s', 'B'),
  ('y', 'B'),
  ('o', 'B'),
  ('p', 'B'),
  ('w', 'B'),
  ('o', 'B'),
  ('f', 'B'),
  ('e', 'B'),
  ('s', 'B'),
  ('y', 'B'),
  ('o', 'B'),
  ('n', 'B'),
  ('è', 'B'),
  ('l', 'B')],
 [('v', 'B'),
  ('e', 'B'),
  ('j', 'B'),
  ('e', 'B'),
  ('t', 'B'),
  ('a', 'B'),
  ('l', 'B')],
 [('r', 'B'),
  ('e', 'B'),
  ('p', 'B'),
  ('i', 'B'),
  ('b', 'B'),
  ('l', 'B'),
  ('i', 'B'),
  ('y', 'B'),
  ('e', 'B')],
 [('m', 'B'), ('a', 'B'), ('s', 'B'), ('o', 'B'), ('n', 'I')],
 [('a', 'B'), ('k', 'B'), ('t', 'B')],
 [('e', 'B'),
  ('w', 'B'),
  ('o', 'B'),
  ('p', 'B'),
  ('e', 'B'),
  ('y', 'B'),
  ('è', 'B'),
  ('n', 'B')],
 [('a', 'B'),
  ('z', 'B'),
  ('è', 'B'),
  ('b', 'B'),
  ('a', 'B'),
  ('y', 'B'),
  ('i', 'B'),
  ('d', 'B

In [23]:
#finding out the vocabulary - this piece of code was taken from Dr.Scannell's notebook
vocabulary = list(set([w for sent in a for (w,t) in sent]))
vocabulary.append('<PAD>')
print(len(vocabulary))

28


In [25]:
#finding out the vocabulary - this piece of code was taken from Dr.Scannell's notebook
tags = list(set([t for sent in a for (w,t) in sent]))
tags.append('<PAD>')
print(tags)
print(a[0])

['B', 'I', '<PAD>']
[('k', 'B'), ('o', 'B'), ('n', 'I'), ('s', 'B'), ('i', 'B'), ('l', 'B'), ('t', 'B'), ('a', 'B'), ('n', 'I')]


In [30]:
#splitting the dataset into training and testing set
X, Y = train_test_split(a,test_size=0.2)

In [32]:
print(len(X))
print(len(Y))
print(X[:10])

10249
2563
[[('a', 'B'), ('t', 'B'), ('a', 'B'), ('n', 'I'), ('d', 'B'), ('a', 'B'), ('n', 'I')], [('r', 'B'), ('e', 'B'), ('y', 'B'), ('a', 'B'), ('s', 'B'), ('i', 'B'), ('r', 'B'), ('e', 'B')], [('p', 'B'), ('a', 'B'), ('n', 'I'), ('s', 'B'), ('y', 'B'), ('o', 'B'), ('n', 'I')], [('p', 'B'), ('a', 'B'), ('t', 'B'), ('o', 'B'), ('l', 'B'), ('o', 'B'), ('g', 'B'), ('i', 'B')], [('t', 'B'), ('o', 'B'), ('t', 'B'), ('a', 'B'), ('l', 'B'), ('m', 'B'), ('a', 'B'), ('n', 'I')], [('s', 'B'), ('w', 'B'), ('i', 'B'), ('s', 'B'), ('i', 'B'), ('d', 'B'), ('e', 'B')], [('p', 'B'), ('r', 'B'), ('e', 'B'), ('f', 'B'), ('e', 'B'), ('r', 'B'), ('a', 'B'), ('n', 'I'), ('s', 'B')], [('r', 'B'), ('e', 'B'), ('k', 'B'), ('i', 'B'), ('l', 'B'), ('e', 'B')], [('k', 'B'), ('o', 'B'), ('n', 'I'), ('t', 'B'), ('a', 'B'), ('j', 'B'), ('y', 'B'), ('e', 'B')], [('s', 'B'), ('a', 'B'), ('n', 'I'), ('k', 'B'), ('s', 'B'), ('y', 'B'), ('o', 'B'), ('n', 'B'), ('e', 'B')]]


In [35]:
#extracting out the list of tagged words
X_tag_word = [i for sent in X for i in sent]
X_tag_word

[('a', 'B'),
 ('t', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('d', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('r', 'B'),
 ('e', 'B'),
 ('y', 'B'),
 ('a', 'B'),
 ('s', 'B'),
 ('i', 'B'),
 ('r', 'B'),
 ('e', 'B'),
 ('p', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('s', 'B'),
 ('y', 'B'),
 ('o', 'B'),
 ('n', 'I'),
 ('p', 'B'),
 ('a', 'B'),
 ('t', 'B'),
 ('o', 'B'),
 ('l', 'B'),
 ('o', 'B'),
 ('g', 'B'),
 ('i', 'B'),
 ('t', 'B'),
 ('o', 'B'),
 ('t', 'B'),
 ('a', 'B'),
 ('l', 'B'),
 ('m', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('s', 'B'),
 ('w', 'B'),
 ('i', 'B'),
 ('s', 'B'),
 ('i', 'B'),
 ('d', 'B'),
 ('e', 'B'),
 ('p', 'B'),
 ('r', 'B'),
 ('e', 'B'),
 ('f', 'B'),
 ('e', 'B'),
 ('r', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('s', 'B'),
 ('r', 'B'),
 ('e', 'B'),
 ('k', 'B'),
 ('i', 'B'),
 ('l', 'B'),
 ('e', 'B'),
 ('k', 'B'),
 ('o', 'B'),
 ('n', 'I'),
 ('t', 'B'),
 ('a', 'B'),
 ('j', 'B'),
 ('y', 'B'),
 ('e', 'B'),
 ('s', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('k', 'B'),
 ('s', 'B'),
 ('y', 'B'),
 ('o', 'B'),
 ('n', 'B'),
 ('e', 'B'),

In [36]:
print(len(X_tag_word))

69793


In [38]:
#extracting out the tokens/important features to analyse. Here, we assume that all the first letter in the words are main features
tokens = [i[0] for i in X_tag_word]
tokens[:3]

['a', 't', 'a']

In [63]:
#find the vocabulary in the data
v = set(tokens)
print(len(V))

27


In [64]:
#find the number of tags
t= set([i[1] for i in X_tag_word])
len(T)

2

In [65]:
print(t)

{'B', 'I'}


In [66]:
# computing P(w/t) and storing them in t x v matrix
tag = len(t)
vocab = len(v)
probab = np.zeros((tag, vocab))

In [67]:
#calculating the emission probability. This piece of code was taken from previous assignment - POS tagging
def emission_probability(word, tag, train_bag = X_tag_word):
    tag_list = [i for i in train_bag if i[1]==tag]
    count_tag = len(tag_list)
    given_tag_list = [i[0] for i in tag_list if i[0]==word]
    given_tag = len(given_tag_list)
    
    return (given_tag, count_tag)

In [74]:
#calculating the transition probability. This piece of code was taken from previous assignment - POS tagging

def transition_probability(t2, t1, train_bag = X_tag_word):
    tags = [i[1] for i in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t += 1
    return (count_t, count_t1)

In [76]:
#finding out transition matrix
matrix = np.zeros((len(t), len(t)), dtype='float32')
for i, t1 in enumerate(list(t)):
    for j, t2 in enumerate(list(t)): 
        matrix[i, j] = transition_probability(t2, t1)[0]/transition_probability(t2, t1)[1]

In [77]:
matrix

array([[0.9044928 , 0.09549151],
       [0.9986866 , 0.00131341]], dtype=float32)

In [78]:
#converting the matrix to a df
df = pd.DataFrame(matrix, columns = list(t), index=list(t))

In [79]:
df

,B,I
B,0.904493,0.095492
I,0.998687,0.001313


In [80]:
print(df.loc['I', :])
print(df.loc['B', :])

B    0.998687
I    0.001313
Name: I, dtype: float32
B    0.904493
I    0.095492
Name: B, dtype: float32


In [81]:
#calculate the Viterbi algorithm. This piece of code was taken from the previous POS tagging assignment.
def Viterbi(words, train_bag = X_tag_word):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = df.loc['B', tag]
                transition_p = df.loc['I',tag]
            else:
                transition_p = df.loc[state[-1], tag]
                
            emission_p = emission_probability(words[key], tag)[0]/emission_probability(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [140]:
length=len(Y)

In [148]:
# list of tagged words
test_tag = [tup for sent in Y for tup in sent]
test_tag

[('m', 'B'),
 ('e', 'B'),
 ('d', 'B'),
 ('y', 'B'),
 ('a', 'B'),
 ('t', 'B'),
 ('i', 'B'),
 ('k', 'B'),
 ('r', 'B'),
 ('e', 'B'),
 ('g', 'B'),
 ('r', 'B'),
 ('e', 'B'),
 ('t', 'B'),
 ('e', 'B'),
 ('i', 'B'),
 ('n', 'B'),
 ('i', 'B'),
 ('i', 'B'),
 ('s', 'B'),
 ('p', 'B'),
 ('a', 'B'),
 ('n', 'B'),
 ('i', 'B'),
 ('k', 'B'),
 ('e', 'B'),
 ('n', 'I'),
 ('t', 'B'),
 ('e', 'B'),
 ('g', 'B'),
 ('r', 'B'),
 ('e', 'B'),
 ('a', 'B'),
 ('z', 'B'),
 ('a', 'B'),
 ('l', 'B'),
 ('y', 'B'),
 ('a', 'B'),
 ('i', 'B'),
 ('l', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('n', 'B'),
 ('m', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('m', 'B'),
 ('i', 'B'),
 ('p', 'B'),
 ('l', 'B'),
 ('e', 'B'),
 ('n', 'B'),
 ('i', 'B'),
 ('t', 'B'),
 ('i', 'B'),
 ('d', 'B'),
 ('k', 'B'),
 ('a', 'B'),
 ('b', 'B'),
 ('ò', 'B'),
 ('n', 'B'),
 ('k', 'B'),
 ('ò', 'B'),
 ('f', 'B'),
 ('l', 'B'),
 ('è', 'B'),
 ('v', 'B'),
 ('p', 'B'),
 ('a', 'B'),
 ('t', 'B'),
 ('i', 'B'),
 ('p', 'B'),
 ('r', 'B'),
 ('i', 'B'),
 ('f', 'B'),
 ('i', 'B'),
 ('s', 'B'),

In [149]:
# tagging the test sentences
start = time.time()
Y_tag = Viterbi(Y)
end = time.time()
difference = end-start

In [186]:
print("Time taken in seconds: ", difference)
print(Y_tag[400])

Time taken in seconds:  57.5466103553772
([('r', 'B'), ('e', 'B'), ('e', 'B'), ('n', 'I'), ('t', 'B'), ('w', 'B'), ('o', 'B'), ('d', 'B'), ('w', 'B'), ('i', 'B')], 'B')


In [159]:
#find out the accuracy 
accuracy_check= [i for i, j in zip(Y_tag, Y)] 

In [160]:
len(accuracy_check)

2563

In [161]:
accuracy = len(accuracy_check)/len(Y_tag)

In [162]:
accuracy

1.0

In [167]:
from google.colab import files
import io

uploaded = files.upload()

Saving test.txt to test.txt


In [174]:
data=pd.read_csv('test.txt',sep='\t',header=None)
data.columns = ['word']
data

,word
0,Oradye
1,bous
2,titan
3,refi
4,netwayè
...,...
1422,Laplen
1423,pwomenad
1424,kore
1425,manti


In [177]:
#convert to lowercase
data['word'] = data['word'].str.lower()
data

,word
0,oradye
1,bous
2,titan
3,refi
4,netwayè
...,...
1422,laplen
1423,pwomenad
1424,kore
1425,manti


In [178]:
test_set=data['word'].to_list()

In [200]:
b = []
for i in test_set:
  list1 = []
  for j in i:
    list1.append((j[0], 'B'))
    if len(j) > 1:
      for k in j[1:]:
        list1.append((k, 'I'))
  b.append(list1)

In [202]:
len(b)

1427

In [201]:
start = time.time()
tag_pred = Viterbi(b)
end = time.time()
difference = end-start

In [205]:
a[703]

[('j', 'B'), ('ò', 'B'), ('j', 'B'), ('è', 'B'), ('t', 'B')]

In [217]:
print(tag_pred[0])
print(difference)

([('o', 'B'), ('r', 'B'), ('a', 'B'), ('d', 'B'), ('y', 'B'), ('e', 'B')], 'B')
60.588390588760376


In [356]:
res=[]
res.append(tag_pred)

In [385]:
tag_pred

[([('o', 'B'), ('r', 'B'), ('a', 'B'), ('d', 'B'), ('y', 'B'), ('e', 'B')],
  'B'),
 ([('b', 'B'), ('o', 'B'), ('u', 'B'), ('s', 'B')], 'B'),
 ([('t', 'B'), ('i', 'B'), ('t', 'B'), ('a', 'B'), ('n', 'B')], 'B'),
 ([('r', 'B'), ('e', 'B'), ('f', 'B'), ('i', 'B')], 'B'),
 ([('n', 'B'),
   ('e', 'B'),
   ('t', 'B'),
   ('w', 'B'),
   ('a', 'B'),
   ('y', 'B'),
   ('è', 'B')],
  'B'),
 ([('g', 'B'),
   ('a', 'B'),
   ('r', 'B'),
   ('a', 'B'),
   ('n', 'B'),
   ('t', 'B'),
   ('i', 'B')],
  'B'),
 ([('g', 'B'), ('a', 'B'), ('y', 'B'), ('a', 'B'), ('k', 'B')], 'B'),
 ([('c', 'B'), ('h', 'B'), ('a', 'B'), ('j', 'B')], 'B'),
 ([('o', 'B'),
   ('f', 'B'),
   ('t', 'B'),
   ('a', 'B'),
   ('l', 'B'),
   ('m', 'B'),
   ('o', 'B'),
   ('l', 'B'),
   ('ò', 'B'),
   ('g', 'B')],
  'B'),
 ([('t', 'B'),
   ('e', 'B'),
   ('k', 'B'),
   ('n', 'B'),
   ('i', 'B'),
   ('k', 'B'),
   ('m', 'B'),
   ('a', 'B'),
   ('n', 'B')],
  'B'),
 ([('a', 'B'), ('d', 'B'), ('o', 'B'), ('n', 'B'), ('i', 'B')], 'B'),
 

In [231]:
s='granchimen'
s1=[]
al=Viterbi(s)
s1.append(al)
s1

[[('g', 'B'),
  ('r', 'B'),
  ('a', 'B'),
  ('n', 'I'),
  ('c', 'B'),
  ('h', 'I'),
  ('i', 'B'),
  ('m', 'B'),
  ('e', 'B'),
  ('n', 'I')]]

In [378]:
k=([[x[1] for x in el] for el in s1])
m=([[x[0] for x in el] for el in s1])

In [382]:
m[0][1]

'r'

In [381]:
k[0]

['B', 'B', 'B', 'I', 'B', 'I', 'B', 'B', 'B', 'I']

In [380]:
m

[['g', 'r', 'a', 'n', 'c', 'h', 'i', 'm', 'e', 'n']]

In [379]:
final=[]
final2=[]
for i in range(0,len(k)):
  for j in range(0,len(k[i])):
        if(k[i][j]=='B' and k[i][j+1]!='I'):
          final.append(m[i][j])
          final.append('-')
        if(k[i][j]=='I'):
          final.extend((m[i][j-1],m[i][j]))
          final.append('-')
  #final2.append(final)
final
print(' '.join(map(str, final)))

g - r - a n - c h - i - m - e n -
